In [35]:
from time import time

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
import time
import torch
import sklearn.datasets
import sklearn.preprocessing
import sklearn.model_selection
import numpy as np
import math
import matplotlib.pyplot as plt
from torchvision.datasets import MNIST
from torchvision.datasets import FashionMNIST as FMNIST
from torchvision.datasets import EMNIST
import torchvision.transforms as transforms
import tensorflow as tf
import random as r
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD, Adagrad, Adadelta, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout, Flatten, Activation, BatchNormalization
import sklearn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
import time
import sys
import math
import numpy as np
import random
import joblib
from tqdm import tqdm_notebook
import copy

import Config
import Dataloader as DL

import matplotlib.pyplot as plt

In [25]:
path="../../Data/"
attacks_types = {
    'normal': 'normal','back': 'dos','buffer_overflow': 'u2r','ftp_write': 'r2l','guess_passwd': 'r2l',
'imap': 'r2l','ipsweep': 'probe','land': 'dos','loadmodule': 'u2r','multihop': 'r2l','neptune': 'dos',
'nmap': 'probe','perl': 'u2r','phf': 'r2l','pod': 'dos','portsweep': 'probe','rootkit': 'u2r','satan': 'probe',
'smurf': 'dos','spy': 'r2l','teardrop': 'dos','warezclient': 'r2l','warezmaster': 'r2l',
}
cols ="""duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,
urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,
num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,
count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,
diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,
dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,
dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate"""
  
columns =[]
for c in cols.split(','):
    if(c.strip()):
       columns.append(c.strip())
print(len(columns))
columns.append('target')
print(len(columns))

attack_categories=["dos","u2r","r2l",'probe','normal']
df = pd.read_csv(path+"kddcup.data_10_percent.gz", names = columns)
df['Attack Type'] = df.target.apply(lambda r:attacks_types[r[:-1]])
del df['target']
df.head()
num_cols = df._get_numeric_data().columns
  
cate_cols = list(set(df.columns)-set(num_cols))
cate_cols.remove('Attack Type')
def getuniquevalues(columnname):
    values={}
    i=0
    for entry in df[columnname]:
        if entry not in values:
            values[entry]=i
            i+=1
    return values
for col in cate_cols:
    df[col]=df[col].map(getuniquevalues(col))
data=df.to_numpy()
Y=df['Attack Type'].map(getuniquevalues('Attack Type'))
Y=Y.to_numpy()
X=data[:,:-1]
print(Y.shape)
print(X.shape)
print(getuniquevalues('Attack Type'))
def normalized(x,y):
    x, x_test, y, y_test = sklearn.model_selection.train_test_split(x, y, shuffle=True)
    scaler = sklearn.preprocessing.Normalizer().fit(x)
    x = scaler.transform(x)
    x_test = scaler.transform(x_test)

    # changes data to pytorch's tensors
    x = torch.from_numpy(x).float()
    y = torch.from_numpy(y).long()
    x_test = torch.from_numpy(x_test).float()
    y_test = torch.from_numpy(y_test).long()
    return x.numpy(), x_test.numpy(), y.numpy(), y_test.numpy(), scaler
xtrain, x_test, ytrain, y_test,scaler= normalized(X,Y)

41
42
(494021,)
(494021, 41)
{'normal': 0, 'u2r': 1, 'dos': 2, 'r2l': 3, 'probe': 4}


In [37]:
# Generate one random vector of desired length and generation type
def generate_vector(vector_length, vector_type, param):
    #Check for Gaussian kernel
    if vector_type == "Gaussian":
        #parse for mu
        mu = param["mu"]
        #parse for sigma
        sigma = param["sigma"]
        #return vector generated from kernel
        return np.random.normal(mu, sigma, vector_length)
    else:
        #all other kernels are currently unsupported
        raise Exception("Vector type %s not recognized. Abort.\n" % vector_type)

def vanilla(param):

    #create basis as list
    basis = []
    #hidden dimension is D
    for _ in range(param["D"]):
        #add one generated vector as a time
        basis.append(generate_vector(param["nFeatures"], param["vector"], param))
    #make basis numpy array
    basis = np.asarray(basis)
    return basis



        #sys.stderr.write(str(self.basis.shape)+"\n")
def updateBasis(basis, param, toChange = None):
    print("Updating basis......")# at the following indices: (None means changing everything)")
    #print(toChange)
    #For each dimension designated to be dropped
    for i in toChange:
        #generate a new ith vector in the basis
        basis[i] = generate_vector(param["nFeatures"], param["vector"], param)
    return

In [38]:
import Config
import sys
import random
import numpy as np
import sklearn
from Config import config, Update_T

# n = e^-(|x|^2/(2std^2)) <- gaussian function
def gauss(x,y,std):
  n = np.linalg.norm(x - y)
  n = n ** 2
  n = n * -1
  n = n / (2 * (std**2))
  n = np.exp(n)
  return n

def poly(x,y,c,d):
  return (np.dot(x,y) + c) ** d

#  dot product/ gauss product/ cos product
def kernel(x,y):
  dotKernel = np.dot
#   gaussKernel = lambda x, y : gauss(x,y,25)
#   polyKernel = lambda x,y : poly(x,y,3,5)
#   cosKernel = lambda x,y : np.dot(x,y) / (np.linalg.norm(x) * np.linalg.norm(y))
  #k = gaussKernel
  #k = polyKernel
  k = dotKernel
  #k = cosKernel
  return k(x,y)

class HD_classifier:

    # Required parameters for the training it supports; will enhance later
    options = ["one_shot", "dropout", "lr"]
    # required opts for dropout
    options_dropout = ["dropout_rate", "update_type"]

    # id: id associated with the basis/encoded data
    def __init__(self, D, nClasses, id):
        self.D = D
        # number of classes
        self.nClasses = nClasses
        # classes
        self.classes = np.zeros((nClasses, D))
        self.counts = np.zeros(nClasses)
        # If first fit, print out complete configuration
        self.first_fit = True
        self.id = id
        self.idx_weights = np.ones((D))
        self.update_cnts = np.zeros((D))
        self.mask = np.ones((D))

    def getClasses(self):
        return self.classes

    def update(self, weight, mask, guess, answer, rate, update_type=Update_T.FULL):
        sample = weight * mask
        self.counts[guess] += 1
        self.counts[answer] += 1
        # update hypervector weights
        if update_type == Update_T.FULL:
            self.classes[guess]  -= rate * weight
            self.classes[answer] += rate * weight
        elif update_type == Update_T.PARTIAL:
            self.classes[guess]  -= rate * sample
            self.classes[answer] += rate * weight
        elif update_type == Update_T.RPARTIAL:
            self.classes[guess]  -= rate * weight
            self.classes[answer] += rate * sample
        elif update_type == Update_T.MASKED:
            self.classes[guess]  -= rate * sample
            self.classes[answer] += rate * sample
        elif update_type == Update_T.HALF:
            self.classes[answer] += rate * weight
            self.counts[guess] -= 1
        elif update_type == Update_T.WEIGHTED:
            self.classes[guess]  -= rate * np.multiply(self.idx_weights, sample)
            self.classes[answer] += rate * np.multiply(self.idx_weights, sample)
        else:
            raise Exception("unrecognized Update_T")

    # update class vectors with each sample, once
    # return train accuracy
    def fit(self, data, label, param = None):

        assert self.D == data.shape[1]

        # Default parameter
        if param is None:
            #set default configuration
            param = Config.config
        for option in self.options:
            if option not in param:
                #if options are missing, set it to default
                param[option] = config[option]
        #if self.first_fit:
        #    sys.stderr.write("Fitting with configuration: %s \n" % str([(k,param[k]) for k in self.options]))

        # Actual fitting

        # handling dropout
        mask = np.ones(self.D)
        if param["masked"]:
            mask = np.copy(self.mask)
        elif param["dropout"]:
            for option in self.options_dropout:
                if option not in param:
                    param[option] = config[option]
            # Mask for dropout
            for i in np.random.choice(self.D, int(self.D * (param["drop_rate"])), replace=False):
                mask[i] = 0

        # fit
        r = list(range(data.shape[0]))
        random.shuffle(r)
        correct = 0
        count = 0
        for i in r:
            sample = data[i] * mask
            assert data[i].shape == mask.shape

            answer = label[i]
            #maxVal = -1
            #guess = -1
            #for m in range(self.nClasses):
            #    val = kernel(self.classes[m], sample)
            #    if val > maxVal:
            #        maxVal = val
            #        guess = m
            vals = np.matmul(sample, self.classes.T)
            guess = np.argmax(vals)
            
            if guess != answer:
                self.update(data[i], mask, guess, answer, param["lr"], param["update_type"])
            else:
                correct += 1
            count += 1
        self.first_fit = False
        return correct / count
    def predict(self, data):

        assert self.D == data.shape[1]

        prediction = []
        # fit
        for i in range(0,data.shape[0]):
            maxVal = -1
            guess = -1
            for m in range(self.nClasses):
                val = kernel(self.classes[m], data[i])
                if val > maxVal:
                    maxVal = val
                    guess = m
            prediction.append(guess)
        return prediction

    # given current classifier value, return:
    # Variance of each dimension across the classes, and
    # The indices in the order from least variance to greatest
    def evaluateBasis(self):
        #normed_classes = self.classes/(np.sqrt(np.asarray([self.counts])).T)
        #variances = np.var(self.classes, axis = 0)
        normed_classes = sklearn.preprocessing.normalize(np.asarray(self.classes), norm='l2')
        variances = np.var(normed_classes, axis = 0) 
        assert len(variances) == self.D
        order = np.argsort(variances)
        return variances, order

    # Some basis are to be update
    def updateClasses(self, toChange = None):
        if toChange is None:
            #self.classes = np.zeros((self.nClasses, self.D))
            self.classes = sklearn.preprocessing.normalize(np.asarray(self.classes), norm='l2', axis = 0)
            self.counts = np.ones(self.nClasses) # An averaged vector is already in
        else:
            for i in toChange:
                self.classes[:,i] = np.zeros(self.nClasses)


In [39]:

# dump basis and its param into a file, return the name of file
def saveEncoded(encoded, labels, id = "", data_type = "unknown"):
    filename = "encoded_%s_%s.pkl" % (id, data_type)
    sys.stderr.write("Dumping data into %s \n"%filename)
    joblib.dump((encoded, labels), open(filename, "wb"), compress=True)
    return filename

# Load basis from a file
def loadEncoded(filename):
    encoded, labels = joblib.load(filename)
    return encoded, labels

# Class: HD_encoder
# Use: take in a basis and a noise flag to create instance, call functions to with data to encode
class HD_encoder:
    def __init__(self, basis, noise=False):
        # set basis to encode with
        self.basis = basis
        # hypervector dimension
        self.D = basis.shape[0]
        #### Noise not implemented yet
        # self.noises = []  
        # if noise:
        #     self.noises = np.random.uniform(0, 2 * math.pi, self.D)
        # else:
        #     self.noises = np.zeros(self.D)

    #encode one vector/sample into a HD vector
    def encodeDatum(self, datum):
        #generate outputs
        encoded = np.matmul(self.basis, datum)
        #nonlinear activation function
        encoded = np.cos(encoded)
        #return results
        return encoded

    # encode data using the given basis
    # noise: default Gaussian noise
    def encodeData(self, data):
        # start = time.time()
        #no matrix multiplication errors
        assert data.shape[1] == self.basis.shape[1]
        # noises = []
        encoded = []

        for i in range(len(data)):
            encoded.append(self.encodeDatum(data[i]))
        # end = time.time()
        #sys.stderr.write("Time spent: %d sec\n" % int(end - start))
        return np.asarray(encoded)

    # Replace basis of the HDE
    def updateBasis(self, basis):
        self.basis = basis

In [40]:


class NeuralHD:
    def __init__(self, classes : int, features : int, dim : int = 400):
        #Configure for hdb, hdc, and hde classes
        self.param=Config.config
        self.param['nClasses'] = classes
        self.param['nFeatures']= features
        #hypervector size
        self.param['D']=dim
        #encoder
        self.hde=None
        #classifier
        self.hdc=None
    def __call__(self, x : torch.Tensor):
        #True iff the model has been trained
        assert self.hde!=None and self.hdc!=None
        #return predicted values
        return self.predict(x)
    def predict(self,x):
        #Get hypervectors for all data points
        trainencoded=self.hde.encodeData(x)
        #return predictions based on similarity to classification hypervectors
        return np.array(self.hdc.predict(trainencoded))
    def fit(self,traindata, trainlabels,
                   epochs,
                   regenloops,  # list of effective dimensions to reach 
                   percentDrop # drop/regen rate 
                    ):
        
        # Initialize basis
        basis = vanilla(self.param)
        # make encoder based on basis
        self.hde = HD_encoder(basis)
        # find encoded training vectors
        trainencoded = self.hde.encodeData(traindata)
        # Initialize classification hypervectors
        self.hdc = HD_classifier(self.param["D"], self.param["nClasses"], 0)

        # calculate amount of dropped dimensions based on percent and original dimension
        amountDrop = int(percentDrop * self.hdc.D)#self.param.D?
        print("Updating times:", regenloops)

        for i in range(regenloops+1): # For each eDs to reach, will checkpoints
            print("regenloop: " + str(i))
            # train for x epochs
            perfect=self.trainreploop(epochs,trainencoded,trainlabels)
            #if its the last regeneration training, stop before doing another dimension drop; stop if 100% accuracy
            if i==regenloops or perfect:
                return #self.hdc,self.hde - unnecessary now that hdc and hde are within a class
            print("regeneration")
            #do the dimension drop and regeneration
            trainencoded=self.regen(basis,amountDrop,traindata)
        return "error","error"
    
    def trainreploop(self,epochs,trainencoded,trainlabels):
        # Do the train 
        for j in range(epochs):
            # do one pass of training
            train_acc = 100 * self.hdc.fit(trainencoded, trainlabels, self.param)
            #Test accuracy used to be in here but I took it out because we dont usually know it
            print("Train: %.2f \t \t Test: "%(train_acc))
            # If accuracy is 100, finish
            if train_acc == 100:
                return True
        return False
    def regen(self,basis,amountDrop,traindata):
        #make order of dimensions according to variaince; also store variances <-- unnecessary?
        var, orders = self.hdc.evaluateBasis()
        #drop dimensions with lowest variance
        toDrop = orders[:amountDrop]
        # print("Variances stats: max %.2f, min %.2f, mean %.2f"%(max(var),min(var),np.mean(var)))
        #update basis by randomizing dimension
        updateBasis(basis,self.param,toDrop)
        # move the new basis into the encoder
        self.hde.updateBasis(basis)
        # normalize previous classes so retrain has enough effect
        self.hdc.updateClasses()
        # get new encoded training data
        return self.hde.encodeData(traindata)    

In [42]:
model=NeuralHD(5,xtrain.shape[1],300)
model.fit(xtrain,ytrain,2,5,.1)

Updating times: 5
regenloop: 0
Train: 98.36 	 	 Test: 
Train: 98.66 	 	 Test: 
regeneration
Updating basis......
regenloop: 1
Train: 98.55 	 	 Test: 
Train: 98.72 	 	 Test: 
regeneration
Updating basis......
regenloop: 2
Train: 98.55 	 	 Test: 
Train: 98.72 	 	 Test: 
regeneration
Updating basis......
regenloop: 3
Train: 98.56 	 	 Test: 
Train: 98.72 	 	 Test: 
regeneration
Updating basis......
regenloop: 4
Train: 98.60 	 	 Test: 
Train: 98.76 	 	 Test: 
regeneration
Updating basis......
regenloop: 5
Train: 98.60 	 	 Test: 
Train: 98.77 	 	 Test: 


In [ ]:
for i in range (0,model.param['nClasses']):
    yhat= model(x_test[y_test==i])
    acc = (yhat==i).mean()
    print('class '+str(i)+' accuracy: ' +str(acc))
    print('points: '+ str(len(yhat)))